In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
serp_api_key = os.getenv("SERPAPI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] =serp_api_key
# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

# Prompt+LLM
基本构成： 
PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话")
model = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG  # 使用 DeepSeek 配置
)
chain = prompt | model

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_68427/4032257909.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(


In [3]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的！这里有一个关于狗熊的经典笑话：\n\n---\n\n**笑话：狗熊的烦恼**\n\n一只狗熊走进图书馆，对管理员说：  \n"我要借一本关于怎么对付‘蜜蜂’的书。"  \n\n管理员好奇地问："为什么是蜜蜂？你不是应该怕猎人吗？"  \n\n狗熊叹了口气："你不懂……上次我偷吃蜂蜜被蛰得满头包，结果猎人看到我肿着脸，直接笑到枪都拿不稳，我这才逃掉的！"  \n\n---\n\n**笑点**：狗熊的逻辑是“蜜蜂比猎人更可怕”，因为猎人本来要抓它，却因为被蜜蜂蛰的滑稽样子笑场了😂  \n\n希望你喜欢！', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 11, 'total_tokens': 151, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'finish_reason': 'stop', 'logprobs': None}, id='run--b7c4faa9-aaa3-46af-af50-366eec06fe72-0')

自定义停止输出符

In [4]:
chain = prompt | model.bind(stop=["\n"])

In [5]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的！来一个关于狗熊的冷笑话：', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'finish_reason': 'stop', 'logprobs': None}, id='run--2442a82f-5488-4d89-9cf0-909b02e4ffa2-0')

兼容openai函数调用的方式

In [6]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "爆梗的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [7]:
chain.invoke({"foo": "男人"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "为什么男人总是喜欢开玩笑？",\n  "punchline": "因为他们觉得自己是个笑话！"\n}', 'name': 'joke'}})

输出解析器

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [9]:
chain.invoke({"foo": "女人"})

'好的，这是一个关于女人的笑话：\n\n有一天，一个女人走进一家宠物店，她对店主说：“我想买一只特别聪明的宠物。”店主笑着回答：“我有一只非常聪明的鹦鹉，它会说七种语言。”女人很兴奋地问：“真的吗？那你能让它给我说说吗？”店主点了点头，然后对鹦鹉说：“鹦鹉，给这位女士说说你会的语言。”鹦鹉立刻回答：“当然！你好，你好，你好，你好，你好，你好，你好！”女人有些失望地说：“这只鹦鹉只会说一句话。”店主笑着说：“是的，但是它会用七种不同的语言说。”'

与函数调用混合使用

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [11]:
chain.invoke({"foo": "女人"})

{'setup': '为什么女人喜欢逛街？', 'punchline': '因为逛街是唯一能让她们花钱时不觉得心疼的事情。'}

In [17]:
#只输出setup
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"foo": RunnablePassthrough()} #使用RunnablePassthrough()跳过prompt
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline") # 定义输出的key
)

In [18]:
#chain.invoke({"foo": "女人"})
#使用RunnablePassthrough()跳过prompt
chain.invoke("女人")

'因为打折时可以一举两得，既能减肥又能增加衣柜！'